In [1]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [5]:
import numpy as np 
import h5py
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input

In [6]:
def load_dataset():
    train_data = h5py.File('train_happy.h5', "r")
    x_train = np.array(train_data["train_set_x"][:]) 
    y_train = np.array(train_data["train_set_y"][:]) 

    test_data = h5py.File('test_happy.h5', "r")
    x_test = np.array(test_data["test_set_x"][:])
    y_test = np.array(test_data["test_set_y"][:]) 
    
    y_train = y_train.reshape((1, y_train.shape[0]))
    y_test = y_test.reshape((1, y_test.shape[0]))
    
    return x_train, y_train, x_test, y_test

In [7]:
X_train, Y_train, X_test, Y_test = load_dataset()

In [21]:
def happy_model(input_shape):
    x_input=Input(input_shape)
    x=ZeroPadding2D((3,3))(x_input)#3,3 means that height and width pad if 2 diff tuples then top bottom in 1 and left right in other
    x=Conv2D(32,(7,7),strides=(1,1),name='conv0')(x)#32 is the number of filters,7,7 is the filter size,1,1 is along the height and the width
    x=BatchNormalization(axis=3,name='bn0')(x)#generally each of the channels in z is normalized
    x=Activation('relu')(x)
    x=MaxPooling2D((2,2),name='max_pool')(x)#this is the maxpool
    x=Conv2D(32,(7,7),strides=(1,1),name='conv02')(x)#32 is the number of filters,7,7 is the filter size,1,1 is along the height and the width
    x=BatchNormalization(axis=3,name='bn02')(x)#generally each of the channels in z is normalized
    x=Activation('relu')(x)
    x=MaxPooling2D((2,2),name='max_pool1')(x)#this is the maxpool
    x=Flatten( )(x)
    x=Dense(1,activation='sigmoid',name='fc1')(x)#1 is the dimentionality of the o/p space
    model=Model(inputs=x_input,outputs=x,name='happymode')
    return model

In [25]:
happymodel1=happy_model(X_train.shape[1:])
happymodel1.compile("adam","binary_crossentropy",["accuracy"])
happymodel1.fit(X_train,Y_train.T,epochs=30,batch_size=50,verbose=1)

Epoch 1/30
600/600 [==============================] - 10s - loss: 0.9479 - acc: 0.6433    
Epoch 2/30
600/600 [==============================] - 11s - loss: 0.3878 - acc: 0.8317    
Epoch 3/30
600/600 [==============================] - 10s - loss: 0.2549 - acc: 0.9267    
Epoch 4/30
600/600 [==============================] - 9s - loss: 0.1959 - acc: 0.9267     
Epoch 5/30
600/600 [==============================] - 10s - loss: 0.1575 - acc: 0.9500    
Epoch 6/30
600/600 [==============================] - 9s - loss: 0.1296 - acc: 0.9633     
Epoch 7/30
600/600 [==============================] - 10s - loss: 0.1066 - acc: 0.9700    
Epoch 8/30
600/600 [==============================] - 9s - loss: 0.0823 - acc: 0.9867     
Epoch 9/30
600/600 [==============================] - 9s - loss: 0.0673 - acc: 0.9850     
Epoch 10/30
600/600 [==============================] - 10s - loss: 0.0566 - acc: 0.9883    
Epoch 11/30
600/600 [==============================] - 11s - loss: 0.0545 - acc: 0.9883  

In [26]:
preds=happymodel1.evaluate(X_test,Y_test.T,batch_size=32,verbose=1,sample_weight=None)
#verbose=0 will show you nothing
#verbose=1 will show you animated progress bar
#verbose=2 will show you slash type of o/p
print("Loss is "+str(preds[0]))
print("Test accuracy is "+str(preds[1]))

150/150 [==============================] - 1s     
Loss is 0.08384069403012594
Test accuracy is 0.9666666706403096
